## Import des libraries

In [1]:
import numpy as np
import torch
import torch.nn as nn
import gdown
import os

## Téléchargement du dataset

In [ ]:
#Téléchargement du dataset
url = 'https://drive.google.com/uc?export=download&id=1iqd7FtqMGstCm6uPZoFbMBNB24_Jlerr'
output = 'dataset.zip'
if not os.path.exists(output):
	gdown.download(url, output, quiet=False)

#Dézippage du dataset

def unzip(zip_file, dest_dir):
	import zipfile
	with zipfile.ZipFile(zip_file, 'r') as zip_ref:
		zip_ref.extractall(dest_dir)

unzip('dataset.zip', './')

## Load le dataset

In [ ]:
X_train = np.load('X_train.npy')
X_clean_train = np.load('X_clean_train.npy')
X_test = np.load('X_test.npy')

print(X_train.shape)
print(X_clean_train.shape)
print(X_test.shape)

## Affichage des données

In [ ]:
import matplotlib.pyplot as plt

# first row show 5 noised images, second row show 5 clean images
for i in range(5):
	plt.subplot(2, 5, i + 1)
	plt.imshow(X_train[i])
	plt.title("Noised")
	plt.axis('off')

	plt.subplot(2, 5, i + 6)
	plt.imshow(X_clean_train[i])
	plt.title("Clean")
	plt.axis('off')

plt.show()

## Batch les données

In [ ]:
batch_size = 20
width = X_train.shape[1]
height = X_train.shape[2]
X_train_reshaped = X_train.reshape(-1, batch_size, width, height)
X_clean_train_reshaped = X_clean_train.reshape(-1, batch_size, width, height)

print(X_train_reshaped.shape)
print(X_clean_train_reshaped.shape)

## Ajouter la dimension de couleur
Pour l'instant, nos images sont de shape (width, height). Les couches de convolution veulent des images de shape (color, width, height). On va donc ajouter une dimension de couleur à nos images.

In [ ]:
X_train_expanded = np.expand_dims(X_train_reshaped, axis=2)
X_clean_train_expanded = np.expand_dims(X_clean_train_reshaped, axis=2)

print(X_train_expanded.shape)
print(X_clean_train_expanded.shape)

## Convertir les données en torch.tensor

In [ ]:
X_train_torch = torch.from_numpy(X_train_expanded).float()
X_clean_train_torch = torch.from_numpy(X_clean_train_expanded).float()

## Créer le modèle

In [ ]:
class Baseline(nn.Module):
	def __init__(self):
		super().__init__()
		self.conv = nn.Conv2d(in_channels=1, out_channels=1, kernel_size=3, padding=1)

	def forward(self, x):
		x = self.conv(x)
		return x


## Boucle d'entraînement

In [ ]:
loss_fn = nn.MSELoss()
model = Baseline()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(10):
	for i in range(len(X_train_torch)):
		optimizer.zero_grad()
		X = X_train_torch[i]
		X_clean = X_clean_train_torch[i]
		X_pred = model(X)
		loss = loss_fn(X_pred, X_clean)
		loss.backward()
		optimizer.step()
		if i % 100 == 0:
			print(f"Epoch {epoch}, iter {i}, loss {loss.item():.2f}")

## Prédiction

In [ ]:
batch_size_test = 14
X_test = X_test.reshape(-1, batch_size_test, width, height)
X_test_expanded = np.expand_dims(X_test, axis=2)
X_test_torch = torch.from_numpy(X_test_expanded).float()

predictions = []
for i in range(len(X_test_torch)):
	X = X_test_torch[i]
	X_pred = model(X)
	predictions.append(X_pred.detach().numpy())

predictions = np.array(predictions)
predictions = predictions.reshape(-1, width, height)

np.save('predictions.npy', predictions)

## Visualiser les résultats

In [ ]:
for i in range(5):
	plt.subplot(2, 5, i + 6)
	plt.imshow(predictions[i])
	plt.title("Clean")
	plt.axis('off')